<center>Национальный исследовательский университет ИТМО<br/>Факультет информационных технологий и программирования<br/>Прикладная математика и информатика</center>

## <center>Математическая статистика</center>
### <center>Отчёт по лабораторной работе №3</center>

<div style="text-align: right"><b>Работу выполнили:</b><br/>Белоцерковченко Андрей M3337<br/>Смирнов Андрей M3337<br/</div>

<center>Санкт-Петербург<br/>2023</center>

In [17]:
import numpy as np
import scipy as sc
from math import sqrt

## Доверительный интервал уровня $1 - \alpha$
### Теория

Есть две выборки $X_1, X_2 \dots X_n \sim N(\mu_1, \sigma_1^2)$ и $Y_1, Y_2 \dots Y_m \sim N(\mu_2, \sigma_2^2)$. Оцениваем $\theta = \cfrac{\sigma_1^2}{\sigma_2^2}$, при этом $\mu_1$ и $\mu_2$ нам известны. 

Первым делом введём слуайную величину $U_1 = \sum\limits_{i = 1}^n(\cfrac{X_i - \mu_1}{\sigma_1})^2$. Т.к. выборка распределена по нормальному закону - мы можем сказать, что $U_1 \sim \chi^2(n)$. Аналогично для $Y_i$ вводим случайную величину $U_2$

По определению распределения Фишера получаем, что $$\cfrac{U_2 / m}{U_1 / n} \sim F(m, n)$$ расписываем отношение и получаем: 
$$\cfrac{n\sum\limits_{j = 1}^m(Y_j - \mu_2)^2}{m\sum\limits_{i = 1}^n(X_i - \mu_1)^2}\tau \sim F(m, n)$$

Теперь мы можем посчитать доверительный ряд: $$P(q_{\frac{\alpha}{2}} \leq \cfrac{n\sum\limits_{j = 1}^m(Y_j - \mu_2)^2}{m\sum\limits_{i = 1}^n(X_i - \mu_1)^2}\tau \leq q_{1 - \frac{\alpha}{2}}) = 1 - \alpha$$

Таким образом, доверительный интервал будет иметь следующий вид: 
$$\tau \in \Bigg[ q_{\frac{\alpha}{2}}\cfrac{m\sum\limits_{i = 1}^n(X_i - \mu_1)^2}{n\sum\limits_{j = 1}^m(Y_j - \mu_2)^2}; q_{1 - \frac{\alpha}{2}}\cfrac{m\sum\limits_{i = 1}^n(X_i - \mu_1)^2}{n\sum\limits_{j = 1}^m(Y_j - \mu_2)^2} \Bigg]$$

### Эксперименты
Напишем класс-генератор выборок:

In [26]:
class NormalDistributionSamples:
    def __init__(self, mu1, sigma1, sz1, mu2, sigma2, sz2, alpha):
        self.X = []
        self.Y = []
        self.l_int = None
        self.r_int = None
        self.outliers_count = 0
        self.mu1 = mu1
        self.sigma1 = sigma1
        self.sz1 = sz1
        self.mu2 = mu2
        self.sigma2 = sigma2
        self.sz2 = sz2
        self.alpha = alpha
        self.q1 = sc.stats.f.ppf(alpha / 2, sz2, sz1)
        self.q2 = sc.stats.f.ppf(1 - alpha / 2, sz2, sz1)
        self.tau = sigma1 / sigma2
    
    def generate_samples(self):
        self.X = sc.stats.norm.rvs(loc=self.mu1, scale=sqrt(self.sigma1), size=self.sz1)
        self.Y = sc.stats.norm.rvs(loc=self.mu2, scale=sqrt(self.sigma2), size=self.sz2)
        
    def count_interval(self):
        a = self.sz2 * sum(map(lambda x: (x - self.mu1) ** 2, self.X))
        b = self.sz1 * sum(map(lambda y: (y - self.mu2) ** 2, self.Y))
        fvalue = a / b
        self.l_int = self.q1 * fvalue
        self.r_int = self.q2 * fvalue
        
    def show_q(self):
        print(sqrt(self.sigma1))
        print(sqrt(self.sigma2))
        print(self.X)
        print(self.Y)
        print(self.q1, self.q2)

In [28]:
sg = NormalDistributionSamples(0, 2, 25, 0, 1, 25, 0.05)
sg.generate_samples()
sg.show_q()

1.4142135623730951
1.0
[-1.12261676  0.79668599 -0.85686263  1.02272439  1.17361091 -1.70500173
 -1.63633982  1.11778102  0.27244849 -1.66967497  3.16883411 -1.96027745
 -0.0101092  -2.21746267 -2.91848485 -0.51933611 -2.99780507 -1.22763072
  0.77249514 -0.43302647 -0.63139754 -0.08000266  1.05224424  0.77619088
 -0.59216862]
[ 0.90906736 -0.37320091 -0.9529387  -0.81226231 -0.91411424 -2.58973188
  0.97246225  0.66059993 -0.65495044 -0.45746022 -1.22588513  0.74683912
 -0.06266284  1.19753521  0.01392363  0.29776271 -0.34943396 -0.49231054
  1.02627548 -0.16542649  0.93269073  1.43189757  0.27998638 -0.13444023
  1.34692512]
0.44836975329719714 2.2303020947471466
